### Criando um resumo de texto

In [1]:
import nltk
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest
from sklearn.metrics import roc_auc_score

In [2]:
link = Request('https://br.guiainfantil.com/materias/natala-arvore-de-natal-conto-de-natal/',
               headers={'User-Agent': 'Mozilla/5.0'})

In [3]:
pagina = urlopen(link).read().decode('utf-8', 'ignore')

In [4]:
soup = BeautifulSoup(pagina, "lxml")
texto = soup.find(id="post-content").text

In [5]:
#dividir o nosso texto em sentenças e depois em palavras
sentencas = sent_tokenize(texto)
palavras = word_tokenize(texto.lower())

In [6]:
#objeto que contera lista de stopwords e pontuações da biblioteca string
#obs: SET (não permite elementos repetidos)
stopwords = set(stopwords.words('portuguese') + list(punctuation))


#remover stopwords (palavras com significado apenas sintatico)
#remover pontuações
palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]

In [7]:
#Quais são as palavras mais importantes: distribuição de frequência
frequencia = FreqDist(palavras_sem_stopwords)

In [8]:
#Score para cada sentença => Quais são as sentenças mais importantes 
#Usando o defaultdict que incluirá uma chave no dicionario sempre que esta não existir 
#no momento da consulta 

sentencas_importantes = defaultdict(int)

In [9]:
#Looping para percorrer todas as sentenças e coletar todas as estatísticas
#e popular o dicionario

#Este codigo popula o dicionário com o índice da sentença (key) 
#e a soma da frequência de cada palavra presente na sentença (value).


for i, sentenca in enumerate(sentencas):
    for palavra in word_tokenize(sentenca.lower()):
        if palavra in frequencia:
            sentencas_importantes[i] += frequencia[palavra]

In [10]:
#De posse dessas informações, podemos selecionar no nosso dicionário as “n” sentenças
#mais importantes para formar o resumo. 
#Usando a funcionalidade nlargest da biblioteca heapq

idx_sentencas_importantes = nlargest(4, sentencas_importantes, sentencas_importantes.get)

In [11]:
#Criando o resumo

for i in sorted(idx_sentencas_importantes):
    print(sentencas[i])

Contar contos no Natal deve ter a finalidade ensinar o verdadeiro significado das festas natalinas para as crianças.
Então o pai da família começou a se perguntar como poderia ganhar dinheiro para passar a noite de Natal dividindo um delicioso peru ao forno com sua família, desfrutando da festa junto ao fogo.
Finalmente, decidiu que já que não vendia nada, que presentearia as árvores de Natal para as pessoas mais pobres que a sua família.
Conto de Natal, en la categoría de Natal en Guiainfantil.com Brasil.


# Extraindo a Frequência de Termos Usando um Bag of words


### Carregando conteudo em português

In [12]:
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer


In [13]:
from nltk.corpus import machado, mac_morpho, floresta, genesis

In [14]:
from nltk.text import Text
ptext1 = Text(machado.words('romance/marm05.txt'), name="Memórias Póstumas de Brás Cubas (1881)")
ptext2 = Text(machado.words('romance/marm08.txt'), name="Dom Casmurro (1899)")
ptext3 = Text(genesis.words('portuguese.txt'), name="Gênesis")
ptext4 = Text(mac_morpho.words('mu94se01.txt'), name="Folha de Sao Paulo (1994)")

In [15]:
def texts():
    print("ptext1:", ptext1.name)
    print("ptext2:", ptext2.name)
    print("ptext3:", ptext3.name)
    print("ptext4:", ptext4.name)
    
texts()

ptext1: Memórias Póstumas de Brás Cubas (1881)
ptext2: Dom Casmurro (1899)
ptext3: Gênesis
ptext4: Folha de Sao Paulo (1994)


### Buscando por palavras

In [16]:
# O método concordance permite ver palavras em um contexto
def busca_palavra(texto, palavra):
    return texto.concordance(palavra)

In [17]:
busca_palavra(ptext1, 'monstro')

Displaying 1 of 1 matches:
oração ; disse - lhe que ela era um monstro , que jamais me tivera amor , que m


In [18]:
busca_palavra(ptext2, 'culpada')

Displaying 2 of 2 matches:
ia o público , se ela deveras fosse culpada , tão culpada como Capitu ? E que m
 se ela deveras fosse culpada , tão culpada como Capitu ? E que morte lhe daria


### Buscando por colocações com algum contexto

https://www.kaggle.com/leandrodoze/examples-from-nltk-book-in-portuguese
    
    https://docs.huihoo.com/nltk/0.9.5/guides/portuguese_en.html

In [19]:
# É possível procurar no texto por colocações significativamente significantes
def colocacoes(texto):
    return texto.collocations()
    

In [20]:
colocacoes(ptext4)

Mundo PáGINA; Sinn Fein; São Paulo; uma bomba; cessação total; Atacada
símbolo; Downing Street; Reino Unido; pessoas morrem; John Major;
Berlusconi nega; Cubanos virão; governo britânico; contra racismo;
para São; virão para; França expulsa; antecipará eleições; Hong Kong;
Partido Unionista


### Questão 

https://stackoverflow.com/questions/59309328/how-to-tokenize-big-text-in-sentences-and-words/59402193#59402193

### Primeira forma de resolver

In [27]:
from nltk.corpus import machado

In [28]:
machado.words()

['Conto', ',', 'Contos', 'Fluminenses', ',', '1870', ...]

In [29]:
text = machado.raw('romance/marm08.txt')

In [30]:
print(text)

Romance, Dom Casmurro, 1899

Dom Casmurro

Texto de referência:

Obras Completas de Machado de Assis,
vol. I,

Nova Aguilar, Rio de
Janeiro, 1994.

 Publicado originalmente
pela Editora Garnier, Rio de Janeiro, 1899.

CAPÍTULO PRIMEIRO

DO TÍTULO

Uma noite destas, vindo da cidade
para o Engenho Novo, encontrei no trem da Central um rapaz aqui do bairro, que
eu conheço de vista e de chapéu. Cumprimentou-me, sentou-se ao pé de mim, falou
da Lua e dos ministros, e acabou recitando-me versos. A viagem era curta, e os
versos pode ser que não fossem inteiramente maus. Sucedeu, porém, que, como eu
estava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele
interrompesse a leitura e metesse os versos no bolso.

 Continue, disse eu acordando.

 Já acabei, murmurou ele.

 São muito bonitos.

Vi-lhe fazer um gesto para
tirá-los outra vez do bolso, mas não passou do gesto; estava amuado. No dia seguinte
entrou a dizer de mim nomes feios, e acabou alcunhando-me Dom Casmurro.

In [31]:
from nltk import word_tokenize, sent_tokenize
text = machado.raw('romance/marm08.txt')
tokenized_words = [word_tokenize(sent) for sent in sent_tokenize(text)]
tokenized_sents = [sent_tokenize(text)]


In [32]:
for sent in tokenized_words:
    for word in sent:
           print(word)
    break

Romance
,
Dom
Casmurro
,
1899
Dom
Casmurro
Texto
de
referência
:
Obras
Completas
de
Machado
de
Assis
,
vol
.


In [33]:
for sent in tokenized_sents:
    for word in sent:
           print(word)
    break

Romance, Dom Casmurro, 1899

Dom Casmurro

Texto de referência:

Obras Completas de Machado de Assis,
vol.
I,

Nova Aguilar, Rio de
Janeiro, 1994.
Publicado originalmente
pela Editora Garnier, Rio de Janeiro, 1899.
CAPÍTULO PRIMEIRO

DO TÍTULO

Uma noite destas, vindo da cidade
para o Engenho Novo, encontrei no trem da Central um rapaz aqui do bairro, que
eu conheço de vista e de chapéu.
Cumprimentou-me, sentou-se ao pé de mim, falou
da Lua e dos ministros, e acabou recitando-me versos.
A viagem era curta, e os
versos pode ser que não fossem inteiramente maus.
Sucedeu, porém, que, como eu
estava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele
interrompesse a leitura e metesse os versos no bolso.
 Continue, disse eu acordando.
 Já acabei, murmurou ele.
 São muito bonitos.
Vi-lhe fazer um gesto para
tirá-los outra vez do bolso, mas não passou do gesto; estava amuado.
No dia seguinte
entrou a dizer de mim nomes feios, e acabou alcunhando-me Dom Casmurro.
Os viz

Tinham passado outros, e ainda outros viriam atrás;
todos iam às suas namoradas.
Era uso do tempo namorar a cavalo.
Relê Alencar:
'Porque um estudante (dizia um dos seus personagens de teatro de 1858) não
pode estar sem estas duas coisas, um cavalo e uma namorada.Relê Álvares de
Azevedo.
Uma das suas poesias é destinada a contar (1851) que residia em
Catumbi, e, para ver a namorada no Catete, alugara um cavalo por três
mil-réis... Três mil-réis!
tudo se perde na noite dos tempos!
Ora, o dandi do cavalo baio
não passou como os outros; era a trombeta do juízo final e soou a tempo; assim
faz o Destino, que é o seu próprio contra-regra.
O cavaleiro não se contentou
de ir andando, mas voltou a cabeça para o nosso lado, o lado de Capitu, e olhou
para Capitu, e Capitu para ele; o cavalo andava, a cabeça do homem deixava-se
ir voltando para trás.
Tal foi o segundo dente de ciúme que me mordeu.
A rigor,
era natural admirar as belas figuras; mas aquele sujeito costumava passar ali,
às tardes; m

 Papai!
papai!
exclamava
Ezequiel.
 Não, não, eu não sou teu pai!
CAPÍTULO CXXXVIII

CAPITU QUE ENTRA

Quando levantei a cabeça, dei com
a figura de Capitu diante de mim.
Eis aí outro lance, que parecerá de teatro, e
é tão natural como o primeiro, uma vez que a mãe e o filho iam à missa, e
Capitu não saía sem falar-me.
Era já um falar seco e breve; a maior parte das
vezes, eu nem olhava para ela.
Ela olhava sempre, esperando.
Desta vez, ao dar com ela, não sei
se era dos meus olhos, mas Capitu pareceu-me lívida.
Seguiu-se um daqueles
silêncios, a que, sem mentir, se pode chamar de um século, tal é a extensão do
tempo nas grandes crises.
Capitu recompôs-se; disse ao filho que se fosse
embora, e pediu-me que lhe explicasse...

 Não há que explicar, disse eu.
 Há tudo; não entendo as tuas
lágrimas nem as de Ezequiel.
Que houve entre vocês?
 Não ouviu o que lhe disse?
Capitu respondeu que ouvira choro e
rumor de palavras.
Eu creio que ouvira tudo claramente, mas confessá-lo seria
perd

### Segunda forma de resolver

In [34]:
from nltk.corpus import machado

In [35]:
import numpy as np 
from nltk.corpus import machado
import nltk

In [36]:


#if you want sent_token of text using sent_tokenize, read textfile in raw form 
raw_text = machado.raw('romance/marm05.txt')


In [37]:
word_token = nltk.word_tokenize(raw_text)
sent_token = nltk.sent_tokenize(raw_text)

print(sent_token[0:2]) # printing 2 sentence, which is tokenized from text
print(word_token[0:20]) # printing 20 words, wich is tokenized from text


['Romance, Memórias Póstumas de Brás Cubas, 1880\n\nMemórias Póstumas de\nBrás Cubas\n\nTexto-fonte:\nObra Completa, Machado de\nAssis,\nRio\nde Janeiro: Editora Nova Aguilar, 1994.', 'Publicado originalmente em\nfolhetins, a partir de março de 1880, na Revista Brasileira.']
['Romance', ',', 'Memórias', 'Póstumas', 'de', 'Brás', 'Cubas', ',', '1880', 'Memórias', 'Póstumas', 'de', 'Brás', 'Cubas', 'Texto-fonte', ':', 'Obra', 'Completa', ',', 'Machado']


#### Funções genéricas

In [38]:
def RemoveStopWords(instancia):
    
    #Remove stopwords no idioma portugues

    stopwords = set(nltk.corpus.stopwords.words('portuguese') )
    palavras = [i for i in instancia.split() if not i in stopwords]
    
    return (" ".join(palavras))


def Limpeza_dados(instancia):
    
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)


def Stemming(instancia):
    
    #reduz o termo ao seu radical, removendo afixos e vogais temáticas.
    
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

    

In [39]:
def tweet_tokenize(instancia):
    #Limpeza de dados de  tweets
    #strip_handles=True - TweetTokenizer irá remover mençoes a usuários do tweet.
    #reduce_len =True - irá reduzir caracteres duplicados
    #preserve_case =  tudo em minúsculo
    
    tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)
    instancia = tweet_tokenizer.tokenize(instancia)
    
    return instancia


# Aplica a função em todos os dados ja processados pela função Preprocessing :
#tweets_token = [tweet_tokenize(i) for i in tweets_process]